In [311]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import json
import sqlite3
import numpy as np
%matplotlib inline

thenums_df = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')
thenums_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0



## I made a function that turned object columns into integer colums ###

I ran this function onto the columns of this dataframe


In [312]:
# This function turns series columns into integers
def make_column_integer(df, series):
    df[series] = df[series].str.replace('$', '')
    df[series] = df[series].str.replace(',', '')
    df[series] = df[series].astype(int)

# Turns all the columns in this dataframe into integers
make_column_integer(thenums_df, 'production_budget')
make_column_integer(thenums_df, 'domestic_gross')
make_column_integer(thenums_df, 'worldwide_gross')


## I then made 2 new columns: 

One column called "total_gross" which was the sum of the domestic and worldwide grossings, and another column called "total_profit" that is the total gross minus the production cost.



In [313]:
thenums_df['total_gross']= (thenums_df['domestic_gross'])+(thenums_df['worldwide_gross'])
thenums_df['total_profit'] = (thenums_df['total_gross']-thenums_df['production_budget'])
thenums_df.sort_values('total_profit', ascending = False, inplace=True)
thenums_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,3536852904,3111852904
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2989973445,2683973445
42,43,"Dec 19, 1997",Titanic,200000000,659363944,2208208395,2867572339,2667572339
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,2726949682,2426949682
33,34,"Jun 12, 2015",Jurassic World,215000000,652270625,1648854864,2301125489,2086125489
...,...,...,...,...,...,...,...,...
480,81,"Dec 31, 2019",Army of the Dead,90000000,0,0,0,-90000000
479,80,"Dec 13, 2017",Bright,90000000,0,0,0,-90000000
341,42,"Jun 14, 2019",Men in Black: International,110000000,3100000,3100000,6200000,-103800000
194,95,"Dec 31, 2020",Moonfall,150000000,0,0,0,-150000000


## I imported a new dataframe from the IMDB database that contained the genres of the movies.

In [314]:
imdb_title_basics = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz')

In [315]:
imdb_title_basics

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,NaN




## I merged the profits dataframe and the title basics dataframe



In [316]:
gandp_df = pd.merge(imdb_title_basics, thenums_df, how='inner', left_on='primary_title', right_on='movie')

In [317]:
# Sorted the values by hightest profit
gandp_df.sort_values('total_profit', ascending=False, inplace=True)

# Drops redundant columns
gandp_df = gandp_df.drop(columns = ['original_title', 'start_year', 'id', 'movie'])

In [318]:
# Drops the duplicated movies from the dataframe
gandp_df = gandp_df.drop_duplicates(subset='primary_title')

In [319]:
#Sets the index to their tconst
gandp_df.set_index('tconst', inplace=True)

In [320]:
#Drops all null values from the genres catagory
gandp_df.dropna(subset=['genres'], inplace=True)

In [321]:
gandp_df.at['tt1775309', 'genres'] = 'Action,Adventure,Fantasy'

## I then broke up the dataframe into sub-dataframes, for each genre we are analyzing

In [322]:
action_movies= gandp_df[gandp_df.genres.str.contains('Action' or 'Adventure')]
drama_movies= gandp_df[gandp_df.genres.str.contains('Drama')]
comedy_movies= gandp_df[gandp_df.genres.str.contains('Comedy')]
horror_movies= gandp_df[gandp_df.genres.str.contains('Horror')]

In [323]:
action_movies

,primary_title,runtime_minutes,genres,release_date,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
tconst,,,,,,,,,
tt1775309,Avatar,93.0,"Action,Adventure,Fantasy","Dec 18, 2009",425000000,760507625,2776345279,3536852904,3111852904
tt4154756,Avengers: Infinity War,149.0,"Action,Adventure,Sci-Fi","Apr 27, 2018",300000000,678815482,2048134200,2726949682,2426949682
tt0369610,Jurassic World,124.0,"Action,Adventure,Sci-Fi","Jun 12, 2015",215000000,652270625,1648854864,2301125489,2086125489
tt0848228,The Avengers,143.0,"Action,Adventure,Sci-Fi","May 4, 2012",225000000,623279547,1517935897,2141215444,1916215444
tt1825683,Black Panther,134.0,"Action,Adventure,Sci-Fi","Feb 16, 2018",200000000,700059566,1348258224,2048317790,1848317790
...,...,...,...,...,...,...,...,...,...
tt1041804,The Nutcracker in 3D,110.0,"Action,Family,Fantasy","Nov 24, 2010",90000000,195459,20466016,20661475,-69338525
tt6745888,The Promise,NaN,Action,"Apr 21, 2017",90000000,8224288,10551417,18775705,-71224295
tt5519340,Bright,117.0,"Action,Crime,Fantasy","Dec 13, 2017",90000000,0,0,0,-90000000


In [324]:
comedy_movies

,primary_title,runtime_minutes,genres,release_date,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
tconst,,,,,,,,,
tt2294629,Frozen,102.0,"Adventure,Animation,Comedy","Nov 22, 2013",150000000,400738009,1272469910,1673207919,1523207919
tt2293640,Minions,91.0,"Adventure,Animation,Comedy","Jul 10, 2015",74000000,336045770,1160336173,1496381943,1422381943
tt2277860,Finding Dory,97.0,"Adventure,Animation,Comedy","Jun 17, 2016",200000000,486295561,1021215193,1507510754,1307510754
tt0435761,Toy Story 3,103.0,"Adventure,Animation,Comedy","Jun 18, 2010",200000000,415004880,1068879522,1483884402,1283884402
tt2283362,Jumanji: Welcome to the Jungle,119.0,"Action,Adventure,Comedy","Dec 20, 2017",90000000,404508916,964496193,1369005109,1279005109
...,...,...,...,...,...,...,...,...,...
tt1817771,Freaks of Nature,92.0,"Comedy,Horror,Sci-Fi","Oct 30, 2015",33000000,70958,70958,141916,-32858084
tt5770620,The Lovers,97.0,"Comedy,Drama,Romance","Mar 13, 2015",35000000,0,53899,53899,-34946101
tt0249516,Foodfight!,91.0,"Action,Animation,Comedy","Dec 31, 2012",45000000,0,73706,73706,-44926294


In [397]:
drama_movies

,primary_title,runtime_minutes,genres,release_date,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
tconst,,,,,,,,,
tt2316801,Beauty and the Beast,112.0,"Drama,Fantasy,Romance","Mar 17, 2017",160000000,504014165,1259199706,1763213871,1603213871
tt6105098,The Lion King,NaN,"Adventure,Animation,Drama","Jun 15, 1994",79300000,421785283,986214868,1408000151,1328700151
tt4283448,Wonder Woman,75.0,"Documentary,Drama,Sport","Jun 2, 2017",150000000,412563408,821133378,1233696786,1083696786
tt1727824,Bohemian Rhapsody,134.0,"Biography,Drama,Music","Nov 2, 2018",55000000,216303339,894985342,1111288681,1056288681
tt1325004,The Twilight Saga: Eclipse,124.0,"Adventure,Drama,Fantasy","Jun 30, 2010",68000000,300531751,706102828,1006634579,938634579
...,...,...,...,...,...,...,...,...,...
tt3864056,The Goldfinch,NaN,Drama,"Sep 13, 2019",40000000,0,0,0,-40000000
tt1014763,Child 44,137.0,"Crime,Drama,History","Apr 17, 2015",50000000,1224330,8004221,9228551,-40771449
tt2935510,Ad Astra,NaN,"Adventure,Drama,Mystery","Sep 20, 2019",49800000,0,0,0,-49800000


In [326]:
horror_movies 

,primary_title,runtime_minutes,genres,release_date,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
tconst,,,,,,,,,
tt1396484,It,135.0,"Horror,Thriller","Sep 8, 2017",35000000,327481748,697457969,1024939717,989939717
tt3300078,The Revenant,80.0,Horror,"Dec 25, 2015",135000000,183637894,532938302,716576196,581576196
tt0816711,World War Z,116.0,"Action,Adventure,Horror","Jun 21, 2013",190000000,202359711,531514650,733874361,543874361
tt6644200,A Quiet Place,90.0,"Drama,Horror,Sci-Fi","Apr 6, 2018",17000000,188024361,334522294,522546655,505546655
tt4779682,The Meg,113.0,"Action,Horror,Sci-Fi","Aug 10, 2018",178000000,145443742,529530715,674974457,496974457
...,...,...,...,...,...,...,...,...,...
tt1403241,Wolves,91.0,"Action,Fantasy,Horror","Nov 14, 2014",18000000,0,94953,94953,-17905047
tt2737304,Bird Box,124.0,"Drama,Horror,Sci-Fi","Dec 13, 2018",19800000,0,0,0,-19800000
tt7043012,Velvet Buzzsaw,113.0,"Horror,Mystery,Thriller","Feb 1, 2019",21000000,0,0,0,-21000000


## I made a list of all possible genres for each movie

In [327]:
# Puts all the genres for each movie into a list
gandp_df['genres'] = list_of_genres

In [328]:
nunique_genres=[]
for x in list_of_genres:
    for genre in x:
        nunique_genres.append(genre)

unique_genres = set(nunique_genres)
unique_genres

{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western'}

## This WebScrapes the taglines from IMDb, which are esentially the the plot lines
### It then organizes the first five taglines into a list for each movie

In [412]:
def get_tags(data):
    no_table = []
    list_of_tags = []
    for tconst in data.index:
        url = f'https://www.imdb.com/title/{tconst}/keywords?ref_=tt_stry_kw'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        
        try:
            table = soup.find_all('table', {'class': 'dataTable'})[0]
            a_tags = [x.find('a') for x in table.find_all('td', {'class': 'soda sodavote'})]
            if len([x.text for x in a_tags]) > 5:
                text = [x.text for x in a_tags][:5]
            else:
                text = [x.text for x in a_tags]
        
            list_of_tags.append(text)
        
        except:
            no_table.append(tconst)
            list_of_tags.append([None])
  
    return list_of_tags

In [413]:
get_tags(comedy_movies.head())

[['magic', 'snowman', 'sister sister relationship', 'ice', 'snow'],
 ['minion', 'invented language', 'one word title', 'villain', 'teddy bear'],
 ['father son relationship',
  'no opening credits',
  'female protagonist',
  'talking whale',
  'fish'],
 ['toy', 'college', 'escape', 'day care', 'evil teddy bear'],
 ['woman with a bare midriff',
  'sexy legs',
  'tattooed trash',
  'legs',
  'bare midriff']]

## I seperated all the movies with no gross earnings into a database called "stream_movies"

In [437]:
stream_movies = gandp_df.loc[gandp_df['total_gross']== 0]
stream_movies = stream_movies.sort_values(by='production_budget', ascending=False)
stream_movies = stream_movies.drop(['domestic_gross','worldwide_gross','total_gross','total_profit'], axis='columns')
stream_movies

,primary_title,runtime_minutes,genres,release_date,production_budget
tconst,,,,,
tt5519340,Bright,117.0,"[Action, Crime, Fantasy]","Dec 13, 2017",90000000
tt7504726,Call of the Wild,NaN,"[Adventure, Animation, Family]","Feb 21, 2020",82000000
tt2479478,The Ridiculous 6,119.0,"[Comedy, Western]","Dec 11, 2015",60000000
tt6924650,Midway,NaN,"[Action, Drama, History]","Nov 8, 2019",59500000
tt7134096,The Rhythm Section,NaN,"[Action, Mystery, Thriller]","Nov 22, 2019",50000000
...,...,...,...,...,...
tt1836212,All Superheroes Must Die,78.0,"[Sci-Fi, Thriller]","Jan 4, 2013",20000
tt1781935,The Ridges,89.0,"[Drama, Horror, Thriller]","Nov 25, 2011",17300
tt3973612,Stories of Our Lives,60.0,[Drama],"Dec 31, 2014",15000


In [ ]:
def best_and_worst(data):
    